<a href="https://colab.research.google.com/github/theAkhileshRai/Projects/blob/master/AssetLvl_Train_Pred_lNN_BinaryClassifierDCP04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder    


from sklearn.metrics import confusion_matrix, classification_report

from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('/content/Old_Normalised_skmin_max.csv',index_col=False)

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,0,0,2019-12-01 00:30:00,1.0,0.602831,0.707424,354.642761,836.740658,0.857422,0.602831,0.715165,0.677025,0.833863,0.865586,Ashland_Unit256,0.0
1,1,1,1,2019-12-01 01:30:00,1.0,0.557169,0.677714,352.424753,837.714329,0.858371,0.557169,0.680772,0.672020,0.836209,0.866544,Ashland_Unit256,0.0
2,2,2,2,2019-12-01 02:30:00,1.0,0.535969,0.669181,352.148637,842.562702,0.981990,0.535969,0.666765,0.678841,0.849176,0.991340,Ashland_Unit256,0.0
3,3,3,3,2019-12-01 03:30:00,1.0,0.624680,0.760470,352.706318,848.715357,0.980724,0.624680,0.760827,0.681220,0.861231,0.990062,Ashland_Unit256,0.0
4,4,4,4,2019-12-01 04:30:00,1.0,0.696433,0.837534,351.392953,851.224013,0.978991,0.696433,0.840353,0.676310,0.866685,0.988312,Ashland_Unit256,0.0


In [ ]:
# df['s_temp_mean*s_pressure_mean'] = df['s_pressure_mean']*df['s_temp_mean']
# df['d_temp_mean*d_pressure_mean'] = df['d_pressure_mean']*df['d_temp_mean']

In [ ]:
# df['s_temp_crank*s_pressure_crank'] = df['s_temp_crank']*df['s_pressure_crank']
# df['d_temp_crank*d_pressure_crank'] = df['d_pressure_crank']*df['d_temp_crank']

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,0,0,2019-12-01 00:30:00,1.0,0.602831,0.707424,354.642761,836.740658,0.857422,0.602831,0.715165,0.677025,0.833863,0.865586,Ashland_Unit256,0.0
1,1,1,1,2019-12-01 01:30:00,1.0,0.557169,0.677714,352.424753,837.714329,0.858371,0.557169,0.680772,0.672020,0.836209,0.866544,Ashland_Unit256,0.0
2,2,2,2,2019-12-01 02:30:00,1.0,0.535969,0.669181,352.148637,842.562702,0.981990,0.535969,0.666765,0.678841,0.849176,0.991340,Ashland_Unit256,0.0
3,3,3,3,2019-12-01 03:30:00,1.0,0.624680,0.760470,352.706318,848.715357,0.980724,0.624680,0.760827,0.681220,0.861231,0.990062,Ashland_Unit256,0.0
4,4,4,4,2019-12-01 04:30:00,1.0,0.696433,0.837534,351.392953,851.224013,0.978991,0.696433,0.840353,0.676310,0.866685,0.988312,Ashland_Unit256,0.0


In [ ]:
# df1 = pd.read_csv('/content/old_normalised_assets-2.csv',index_col=False)
# df2 = pd.read_csv('/content/new_normalised_assets-2.csv',index_col=False)

In [ ]:
# 'Clark county_6', 'RJ booster_1', 'Strawberry_3.csv'

In [9]:
df['Asset'].unique()   

array(['Ashland_Unit256', 'Center station_C2', 'Crossbar_Unit1',
       'El Dorado_Unit', 'Goliad_Unit1100', 'HobitLP_Unit102',
       'Hobit_Unit2', 'Katy_Unit1116', 'Katy_Unit1117', 'LOHT_North',
       'La Porte_K11A', 'RJ_Unit1(1).csv', 'Lightning_Unit2 (2)(1).csv'],
      dtype=object)

In [10]:
df['Asset'].unique()    

array(['Ashland_Unit256', 'Center station_C2', 'Crossbar_Unit1',
       'El Dorado_Unit', 'Goliad_Unit1100', 'HobitLP_Unit102',
       'Hobit_Unit2', 'Katy_Unit1116', 'Katy_Unit1117', 'LOHT_North',
       'La Porte_K11A', 'RJ_Unit1(1).csv', 'Lightning_Unit2 (2)(1).csv'],
      dtype=object)

In [11]:
len(df['Asset'].unique() )

13

In [ ]:
# 'Clark county_6') | ('La Porte_K11A') |\
#              ('Strawberry_Unit1') | ('RJ booster_1') | ('El Dorado_Unit'

In [13]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,0,0,2019-12-01 00:30:00,1.0,0.602831,0.707424,354.642761,836.740658,0.857422,0.602831,0.715165,0.677025,0.833863,0.865586,Ashland_Unit256,0.0
1,1,1,1,2019-12-01 01:30:00,1.0,0.557169,0.677714,352.424753,837.714329,0.858371,0.557169,0.680772,0.672020,0.836209,0.866544,Ashland_Unit256,0.0
2,2,2,2,2019-12-01 02:30:00,1.0,0.535969,0.669181,352.148637,842.562702,0.981990,0.535969,0.666765,0.678841,0.849176,0.991340,Ashland_Unit256,0.0
3,3,3,3,2019-12-01 03:30:00,1.0,0.624680,0.760470,352.706318,848.715357,0.980724,0.624680,0.760827,0.681220,0.861231,0.990062,Ashland_Unit256,0.0
4,4,4,4,2019-12-01 04:30:00,1.0,0.696433,0.837534,351.392953,851.224013,0.978991,0.696433,0.840353,0.676310,0.866685,0.988312,Ashland_Unit256,0.0


In [14]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'time', 'cyl',
       's_temp_crank', 'd_temp_crank', 's_pressure_crank', 'd_pressure_crank',
       'rpm_crank', 's_temp_mean', 'd_temp_mean', 's_pressure_mean',
       'd_pressure_mean', 'rpm_mean', 'Asset', 'Binary'],
      dtype='object')

In [15]:
df['Binary'] = df['Binary'].fillna(0)
df = df.fillna(0)
#df1['multi_class'] = df1['multi_class'].fillna(0)

In [16]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,Binary
0,0,0,0,2019-12-01 00:30:00,1.0,0.602831,0.707424,354.642761,836.740658,0.857422,0.602831,0.715165,0.677025,0.833863,0.865586,Ashland_Unit256,0.0
1,1,1,1,2019-12-01 01:30:00,1.0,0.557169,0.677714,352.424753,837.714329,0.858371,0.557169,0.680772,0.672020,0.836209,0.866544,Ashland_Unit256,0.0
2,2,2,2,2019-12-01 02:30:00,1.0,0.535969,0.669181,352.148637,842.562702,0.981990,0.535969,0.666765,0.678841,0.849176,0.991340,Ashland_Unit256,0.0
3,3,3,3,2019-12-01 03:30:00,1.0,0.624680,0.760470,352.706318,848.715357,0.980724,0.624680,0.760827,0.681220,0.861231,0.990062,Ashland_Unit256,0.0
4,4,4,4,2019-12-01 04:30:00,1.0,0.696433,0.837534,351.392953,851.224013,0.978991,0.696433,0.840353,0.676310,0.866685,0.988312,Ashland_Unit256,0.0


In [17]:
df.shape

(60414, 17)

In [18]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'time', 'cyl',
       's_temp_crank', 'd_temp_crank', 's_pressure_crank', 'd_pressure_crank',
       'rpm_crank', 's_temp_mean', 'd_temp_mean', 's_pressure_mean',
       'd_pressure_mean', 'rpm_mean', 'Asset', 'Binary'],
      dtype='object')

In [19]:
X_columns = [ 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',\
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',\
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean']

In [20]:
len(X_columns)

11

In [21]:
X = df[X_columns]

In [22]:
X.shape

(60414, 11)

# build  with mean

In [23]:
X

,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean
0,1.0,0.602831,0.707424,354.642761,836.740658,0.857422,0.602831,0.715165,0.677025,0.833863,0.865586
1,1.0,0.557169,0.677714,352.424753,837.714329,0.858371,0.557169,0.680772,0.672020,0.836209,0.866544
2,1.0,0.535969,0.669181,352.148637,842.562702,0.981990,0.535969,0.666765,0.678841,0.849176,0.991340
3,1.0,0.624680,0.760470,352.706318,848.715357,0.980724,0.624680,0.760827,0.681220,0.861231,0.990062
4,1.0,0.696433,0.837534,351.392953,851.224013,0.978991,0.696433,0.840353,0.676310,0.866685,0.988312
...,...,...,...,...,...,...,...,...,...,...,...
60409,1.0,0.560897,0.482793,0.585013,0.517756,0.000000,0.560897,0.615550,0.597892,0.563024,0.000000
60410,1.0,0.633780,0.514327,0.569769,0.512984,0.000000,0.633780,0.654050,0.579155,0.558810,0.000000
60411,1.0,0.627929,0.512554,0.578790,0.520432,0.000000,0.627929,0.651635,0.590008,0.566225,0.000000
60412,1.0,0.541300,0.484754,0.578827,0.525011,0.000000,0.541300,0.617035,0.591515,0.570471,0.000000


In [29]:
y_true = df.iloc[:,16]

In [30]:
y_true.unique()

array([0., 1.])

In [31]:
#seed = 42
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_size,shuffle= True)

In [32]:
# X_val, X_test, y_val, y_test = train_test_split(X_Test, y_Test, test_size=0.75)

In [33]:
from collections import Counter

In [34]:
print(X_train.shape,y_train.shape)

(51351, 11) (51351,)


In [35]:
len(X_test)

9063

In [ ]:
#len(X_val)

In [36]:
Counter(y_train)

Counter({0.0: 38931, 1.0: 12420})

64->16->4->1

In [37]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 11.
        self.layer_1 = nn.Linear(11, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_3 = nn.Linear(64, 64)
        self.layer_4 = nn.Linear(64, 10)
        self.layer_5 = nn.Linear(10, 4)
        self.layer_out = nn.Linear(4, 1) 
        xavier_uniform_(self.layer_out.weight)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.batchnorm4 = nn.BatchNorm1d(10)
        self.sig = nn.Sigmoid()

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.dropout(x)
        x = self.layer_out(x)
        x = self.sig(x)
        
        return x

In [38]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [39]:
X_test

array([[ 1.24998878,  0.22031633, -0.41556245, ...,  0.73494253,
         0.67999446,  1.2634031 ],
       [-1.06544554,  0.7425489 ,  0.84930203, ..., -0.98583619,
         0.53797465, -1.04995457],
       [-1.06544554,  1.41681625,  1.25990732, ...,  0.96353076,
         1.17025495,  1.48598057],
       ...,
       [-1.06544554,  2.78694672,  1.19660408, ...,  0.85602244,
        -0.034931  ,  1.48343749],
       [-1.06544554, -1.06666353, -1.87267241, ...,  1.36455469,
         0.69085023, -0.28382825],
       [-1.06544554,  1.21175087,  0.72369733, ..., -1.1438188 ,
         0.12635686, -1.04995457]])

In [40]:
EPOCHS = 10
BATCH_SIZE = 75
LEARNING_RATE = 0.0792

In [41]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

y_train = y_train.values
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
# ## val data    
# class valData(Dataset):
    
#     def __init__(self, X_data):
#         self.X_data = X_data
        
#     def __getitem__(self, index):
#         return self.X_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)

test_data = testData(torch.FloatTensor(X_test))
# val_data = valData(torch.FloatTensor(X_val))

In [42]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(dataset=val_data, batch_size=1)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [44]:
model = binaryClassification()
model.to(device)

binaryClassification(
  (layer_1): Linear(in_features=11, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sig): Sigmoid()
)

In [45]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [46]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [47]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        #X_batch = X_batch
  
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

In [48]:
y_pred_list = []
pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        pred_list.append(y_test_pred.cpu().numpy())
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [49]:
train_data = pd.DataFrame({'Values': y_pred_list, 'Predictions': pred_list})

In [50]:
train_data.to_csv('Train Predictions.csv')

In [51]:
confusion_matrix(y_test, y_pred_list)

array([[6736,  127],
       [ 682, 1518]])

# Old Assets

In [53]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      6863
         1.0       0.92      0.69      0.79      2200

    accuracy                           0.91      9063
   macro avg       0.92      0.84      0.87      9063
weighted avg       0.91      0.91      0.91      9063



In [54]:
import pickle
filename = 'binarynn_dcp_model.pkl'

In [55]:
torch.save(model.state_dict(),filename)


In [56]:
model.load_state_dict(torch.load(filename))

<All keys matched successfully>

In [57]:
model.eval()

binaryClassification(
  (layer_1): Linear(in_features=11, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sig): Sigmoid()
)